In [ ]:
# !pip install transformers==4.43.2
!pip install bitsandbytes==0.43.3
!pip install -U bitsandbytes
!pip install accelerate==0.33.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.1 MB/s eta 0:00:00
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.43.3
    Uninstalling bitsandbytes-0.43.3:
      Successfully uninstalled bitsandbytes-0.43.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.34.2
    Uninstalling accelerate-0.34.2:
      Successfully uninstalled accelerate-0.34.2


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from huggingface_hub import login

# HF_TOKEN = os.getenv("HF_TOKEN")
login(token="")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
def load_model(base_model, torch_dtype=torch.bfloat16, use_quantization=False, quantization_config=None):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    attn_implementation = "eager"

    if use_quantization:
        if quantization_config is None:
            bnb_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype=torch_dtype,
                bnb_4bit_use_double_quant=True,
            )
    else:
        bnb_config = None

    model = AutoModelForCausalLM.from_pretrained(
        base_model,
        quantization_config=bnb_config,
        torch_dtype=torch_dtype,
        device_map="auto",
        attn_implementation=attn_implementation
    )

    if not use_quantization:
        model.to(device)

    return model

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16
model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [ ]:
device

'cuda'

In [ ]:
model = load_model(model_name, use_quantization=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)
config = model.config


print(f"Memory Allocated: {round(torch.cuda.memory_allocated() / (1024 ** 2), 3)}GB")
print(f"Memory Reserved: {round(torch.cuda.memory_reserved() / (1024 ** 2), 3)}GB")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Memory Allocated: 5438.869GB
Memory Reserved: 5528.0GB


### 프롬프트 작성하기

In [ ]:
example = """
'''
이 제품은 가격대비 가성비가 좋고, 토핑도 풍부해서 맛있다는 소비자들의 평이 많아요. 집에서 간편하게 끓여 먹을 수 있어서 편리하고, 양도 푸짐해서 가족 모두에게 적합해요. 다만, 유통기한이 짧아서 버리는 경우가 있었다는 의견도 있어요.
'''

'''
가격 대비 만족스러운 제품이에요. 순두부찌개나 차돌박이와 함께 먹으면 더 맛있어요. 밀키트의 장점을 모두 갖춘 제품으로 편리해요. 고기가 푸짐하고 씹는 재미도 있어서 두 명이 함께 먹기 좋아요. 신선한 재료로 만들어져 있어 매번 자주 구매하는 제품입니다. 백반집 수준의 퀄리티에 매우 만족하실 거예요.
'''

'''
가격 대비로 만족하고, 집에서 간편하게 전문점 맛을 느낄 수 있어요. 청국장이 특히 좋다는 의견도 많아요. 양도 푸짐하고, 냄새도 거의 없어서 신선한 재료를 사용한 것 같아요.
'''

'''
신선하고 좋은 제품이라고 하셔서 만족스러우시다면, 가성비가 좋아서 자주 구매하시는 고객님들이 많으실 것 같아요. 집에 추가 재료를 넣어 맛을 더해도 좋다는 팁도 있어요. 대학생 아이들이 혼자 먹기에도 편리하다는 이야기도 있네요. 스팸과 김치를 함께 끓여먹으면 더 맛있다는 소중한 정보도 함께 전해드릴게요!
'''

'''
요리하기 편하고 빠르게 먹을 수 있는 제품이에요. 냄비를 바로 끓일 수 있어서 편리하고, 12인용으로 적당한 양이에요. 혼자 먹기에도 적당하고, 맛있어서 가게에서 사먹는 것 같아요. 맵지 않아서 부드럽고, 부재료를 더 넣어서 더 맛있게 먹을 수 있어요. 만족스러운 제품이라 재구매 의사가 있습니다!
'''
"""

In [ ]:
example_recipe = """
'''
이렇게 활용하면 좋아요 : 만둣국, 갈비탕
추가하면 좋아요 : 물, 대파, 기름 적은 소고기, 밥
조리 방법 : 물을 먼저 넣고 팔팔 끓이다가 소고기를 넣어주세요. 마지막에 밥을 말아 드세요.
더 좋은 이유 : 냄비에 한 번에 넣고 끓이면 끝
'''

'''
이렇게 활용하면 좋아요 : 해물 오뎅탕
추가하면 좋아요 : 해산물, 오뎅 사리, 무, 대파, 후추, 소금
'''

'''
추가하면 좋아요 : 샤브샤브소고기, 차돌박이, 칼국수면
조리 방법 : 다 먹고 칼국수면을 넣어서 끓여 드세요.
더 좋은 이유 : 전자레인지에 돌리기만 해도 완성
'''

'''
이렇게 활용하면 좋아요 : 김치찌개, 라면
추가하면 좋아요 : 김치, 라면사리, 각종 야채
조리 방법 : 김치를 많이 넣고, 각종 야채는 조금만 추가해주세요.
'''

"""

### 데이터 적용하기(상품별)

In [ ]:
import pandas as pd

# Set pandas options to display full content of DataFrame
pd.set_option('display.max_colwidth', None)  # No truncation of column width
pd.set_option('display.max_rows', None)      # No truncation of rows
pd.set_option('display.max_columns', None)   # No truncation of columns

origin1 = pd.read_csv("/content/drive/MyDrive/감정 태깅/50개씩_감정태깅(최종).csv")
origin1.head()

,product_id,product_type,date,help_num,cleaned_Kiwi_review,sentiment
0,5013198,한우 도가니탕,2023-11-30,2,하트찜꽁 쫀득한 도가니탕 맛있네요 국 간은 싱거운데 소금 간 하면 딱이에요,positive
1,5013198,한우 도가니탕,2023-09-25,2,일부러 내용 청 가물을 보고 샀습니다 가장 청 가물과 나트륨이 적었구요 가격은 음 착하지는 않습니다 추가 구매해서 사 먹을 맛은 아닌 걸로 제가 도가니 매니아가 아니라 그런 걸까요 아니면 후기를 좋은 말만 쓰면 뭐가 있는 건가요 컬리 초보자라 궁금하군요,positive
2,5013198,한우 도가니탕,2023-01-24,2,기름 너무 많아서 느끼함,negative
3,5013198,한우 도가니탕,2024-06-03,1,믿고 먹는 설성 목장이예요 무 항생제인데 도가니도 꽤 들어 있고 가격 대비 좋네요,positive
4,5013198,한우 도가니탕,2024-03-05,1,설성 목장은 소금과 첨가물 안 들어 있어서 좋아요,positive


In [ ]:
def extract_assistant_text(outputs):
    return tokenizer.decode(outputs[0], skip_special_tokens=True).split("assistant")[-1].strip()

In [ ]:
# Specify the product_id you want to test
test_product_id = 5013198  # Replace with the product_id you want to test

# Filter the data for the specific product_id
filtered_data = origin1[origin1['product_id'] == test_product_id]

# Group by sentiment and concatenate the reviews for this product_id
grouped_data = filtered_data.groupby('sentiment')['cleaned_Kiwi_review'].apply(' '.join).reset_index()
# Filter out rows where sentiment is 'neutral'
grouped_data = grouped_data[grouped_data['sentiment'] != 'neutral']
grouped_data

,sentiment,cleaned_Kiwi_review
0,negative,기름 너무 많아서 느끼함 도가니탕 관절 위해 먹어 보려구요 고기가 까매서 왜 그럴지 궁금 엄마 주문해 드렸는데 도가니보다 부속물이 많고 기름 심해서 버렸다고 열자마자 놀랬네요\n 두 개 주문했는데 하나는 멀쩡한데 하나는 누가 일부러 뜯어 놓은 거마냥 뭔가요\n저걸 어떻게 먹어요 칼로 난도질해 놓은 것도 아니고 시아버님 사 드렸는데 별로라고 하시네요 첨에는 안에 내용물이 많이 들어 있어서 도가니가 많이 들어 있는 줄 알았는데 먹다 보니\n고기 몇 점과 지방 덩이네요 ㅠ\n저는 입이 짧아 지방같이 물컹거리는\n부분은 먹지 않는데 \n재구매는 하지 않을 듯요 누린내가 너무 심합니다 이런 냄새를 참나 갑자기 묽어졌어요 \n혹시 물량이 많아졌나요 \n진한 맛이 나라 17000원에 한 끼 아깝지 않게 먹었는데\n이렇게 물 탄 맛이면 앞으로는 시킬 이유가 없겠어요\n왜 맛이 묽어졌는지 궁금합니다 혈압이 살짝 왔다갔다 하는 짝꿍을 위해 설 성 도가니나 설렁탕 나트륨 낮아 주문해요
2,positive,하트찜꽁 쫀득한 도가니탕 맛있네요 국 간은 싱거운데 소금 간 하면 딱이에요 일부러 내용 청 가물을 보고 샀습니다 가장 청 가물과 나트륨이 적었구요 가격은 음 착하지는 않습니다 추가 구매해서 사 먹을 맛은 아닌 걸로 제가 도가니 매니아가 아니라 그런 걸까요 아니면 후기를 좋은 말만 쓰면 뭐가 있는 건가요 컬리 초보자라 궁금하군요 믿고 먹는 설성 목장이예요 무 항생제인데 도가니도 꽤 들어 있고 가격 대비 좋네요 설성 목장은 소금과 첨가물 안 들어 있어서 좋아요 입에 짝 달라붙는 맛\n도가니탕 중 최고입니다 국물 진하고 맛 잇어요 맛있어요 다만 고기 살이 까매요 설성 목장 도가니탕 비싸도 제대로 집에서 요리된 듯 맛이라 꾸준하게 주문해요 도가니탕에 만두국 끓임 최고예요 마켓컬리에서 구입해서 먹은 도가니탕 중에 국물 맛은 제일 맛있어요 근데 제가 구입해서 먹은 제품에 비해 고기에 기름이 많아요\n그것만 개선되면 자주 이용할 거 같아요 매번 구매해서 맛있게 먹고 있습니다 도가니를 좋아하는 딸을 위해 샀어요 ㅋ 도가니가 양이 적긴한데 파 송송에 후추 넣어서 먹으니 맛나요 컬리 제품 자주 구입해요 후기 좋아 구입했어요\n맛은 어떤 양념도 첨가하지 않은 순수한 탕 맛이라 맘에 들지만 고기는 많이 우려서인지 맛이 안 느껴지고 도가니도 조금밖에 안 들어가서 아쉬웠어요 엄마가 제일 좋아하는 도가니탕입니다 여러 브랜드 제품 드려 봐도 이게 제일 부드럽고 진하면서 맛있다고 하시네요 가격대가 있어도 이 제품만 구매하게 되요 설성 목장 도가니탕이 최고입니다 도가니탕 좋아하시는 팔순 훌쩍 넘기신 시아버님이 제가 주문해 드린 도가니탕이 제일 맛있다고 하셔서 주기적으로 보내 드리고 있어요\n홈쇼핑 판매하는 거랑은 비교 불가 입니다 고기와 도가니가 적당히 들어 있어요\n양이 적은 사람은 둘이서 먹어도\n부족하지 않아요 설성 목장 한우 도가니탕\n입맛 없을 때 좋아요 설성 목장 한우 도가니탕\n고기도 도가니도 많아 넘 좋아요 품질이 좋은 이유는 육수가 맑고 구수하다는 것입니다 가격이 비싸서 좀 아쉽습니다 도가니탕은 여기가 찐 입니다 양은 작아도 맛은 좋습니다 후기가 좋아서 구매 혼자 사시는 어른 보내 드렸어요 집에서 한 거 같아서 자주 주문해요\n좋아요 할인해서 주문 설성 목장 도가니탕 한우라 좋고 국물이 깔끔해요 설성 목장 한우 도가니탕 고기도 듬뿍\n도가니도 많고 국물도 깔끔 비싸도 최고네요 설성 목장 한우 도가니탕 너무 맛있어요 간편해서 좋아요\n맛도 좋아요 믿고 먹는 설성 목장입니다 뭘 먹어도 실패가 없네요 사골 스틱도 먹어 보니까 너무 편하고 맛있던데 더 다양한 상품을 컬리에서 구매할 수 있었으면 좋겠어요 홈쇼핑으로 처음 접했는데 낱개로 살 수 있어 좋아요 부모님이 제일 좋아하십니다 자극적이지 않고 판매하는 특유의 그런 맛이 안 난다고 좋아하세요 세일 자주 해 주세요 설성 목장 도가니탕 정말 맛있었요 도가니탕으로 떡국 끓여 먹었는데 맛있어요


In [ ]:
summary_data = []

# Loop through the grouped data (which is already filtered by a single product_id)
for _, row in grouped_data.iterrows():
    sentiment = row['sentiment']
    combined_reviews = row['cleaned_Kiwi_review']

    # Prepare the initial summarization prompt
    summary_prompt = f"""
    당신은 상품에 대한 고객의 리뷰를 요약해서 소비자에게 정보를 제공하는 소비자 분석가입니다.

    **지시사항:**
    리뷰: {combined_reviews}\n
    이 리뷰들을 요약해주세요.
    많이 나오는 내용을 중심으로 요약해주세요.
    요약한 문장을 한 번 더 매끄러운 문장으로 만들어 주세요.

    **제한사항:**
    브랜드 이름은 넣지 마세요.
    리뷰, 요약이라는 단어를 삭제해 주세요.

    **예시:**
    {example}
    """
    inputs = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n{summary_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>\n{combined_reviews}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"

    # First pass: Generate the initial summary
    outputs = model.generate(**tokenizer(inputs,
                             return_tensors="pt").to(device),
                             max_new_tokens=200,
                             pad_token_id=tokenizer.eos_token_id,
                             repetition_penalty=1.2,
                             min_p = 0.05
                             )
    initial_summary = extract_assistant_text(outputs)

    # Second pass: Condense the summary into one sentence
    rephrase_prompt = f"""
    당신은 문장을 간결하게 만드는 전문가입니다.

    **지시사항:**
    다음 문장을 100~150자 사이로 요약해주세요.
    말투는 존댓말이어야 합니다.
    리뷰, 요약이라는 단어를 삭제해 주세요.:
    {initial_summary}
    """
    rephrase_inputs = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n{rephrase_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>\n{initial_summary}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"

    # Generate the final rephrased summary
    rephrase_outputs = model.generate(**tokenizer(rephrase_inputs, return_tensors="pt").to(device), max_new_tokens=200, pad_token_id=tokenizer.eos_token_id)
    final_summary = extract_assistant_text(rephrase_outputs)

    # Save the final summary along with sentiment
    summary_data.append([test_product_id, sentiment, final_summary])

# Create a DataFrame for the summarized data
summary_df = pd.DataFrame(summary_data, columns=['product_id', 'sentiment', 'summary'])

# Display the result
summary_df

,product_id,sentiment,summary
0,5013198,negative,"소비자들은 제품의 기름이 많아 불만족스럽고, 구입 직후 냄새가 심하여 불편했습니다. 또한, 일부 제품들은 이미 손상된 상태로 판매되는 것 같았고, 진한 맛보다는 얇은 맛이 강조되었습니다."
1,5013198,positive,"소비자들은 이 제품을 만족스럽게 생각합니다. 특히 '청 가물'과 '무 항생제'를 비롯한 많은 장점을 높이 평가했습니다. 도가니탕 중에서는 최고라고 말하는 분도 있었고, 국물이 진하고 맛있게 끓여내는 점을 특히 좋게 생각했습니다. 일부는 가격이 높다고 불만을 표명했지만, 다른 분들은 합리적인 가격이며, 가정에서도 저렴하게 요리하여 먹을 수 있다는 장점을 지적했습니다. 또한 도가니가 적절하게 들어감으로써 양이 충분하며, 생강이나 후추 등 다른 조미료를 첨가함으로써 더욱더 맛있게 먹을 수 있다는 점을 알려왔습니다."


### 데이터 적용하기(문장별)

In [ ]:
import pandas as pd

# Set pandas options to display full content of DataFrame
pd.set_option('display.max_colwidth', None)  # No truncation of column width
pd.set_option('display.max_rows', None)      # No truncation of rows
pd.set_option('display.max_columns', None)   # No truncation of columns

origin2 = pd.read_csv("/content/drive/MyDrive/딥다이브 프로젝트/카테고리_태깅된_문장(최종).csv")
origin2.head()

,product_id,review_index,review,category
0,5013198,0,하트찜꽁 쫀득한 도가니탕 맛있네요,맛/향/풍미
1,5013198,0,국 간은 싱거운데 소금 간 하면 딱이에요,맛/향/풍미
2,5013198,0,하트찜꽁 쫀득한 도가니탕 맛있네요,맛/향/풍미
3,5013198,0,국 간은 싱거운데 소금 간 하면 딱이에요,맛/향/풍미
4,5013198,1,가격은 음 착하지는 않습니다,가격


In [ ]:
def extract_assistant_text(outputs):
    return tokenizer.decode(outputs[0], skip_special_tokens=True).split("assistant")[-1].strip()

In [ ]:
# Specify the product_id you want to test
test_product_id = 5013198  # Replace with the product_id you want to test

# Filter the data for the specific product_id
filtered_data = origin2[origin2['product_id'] == test_product_id]

# Group by category and concatenate unique reviews for this product_id
grouped_data2 = filtered_data.groupby('category')['review'].apply(lambda x: ' '.join(x.drop_duplicates())).reset_index()

# Create a separate dataframe for rows where 'category' is '활용 방법'
grouped_recipe = grouped_data2[grouped_data2['category'] == '활용 방법']

# For the remaining data (excluding '활용 방법'), filter rows where the 'review' column has 10 or more words
grouped_data2 = grouped_data2[grouped_data2['category'] != '활용 방법']
grouped_data2 = grouped_data2[grouped_data2['review'].apply(lambda x: len(x.split()) >= 10)]

# Display the filtered results
print("Grouped Recipe DataFrame:")
display(grouped_recipe)

print("Filtered Grouped DataFrame (excluding '활용 방법'):")
display(grouped_data2)

Grouped Recipe DataFrame:


,category,review
5,활용 방법,도가니탕에 만두국 끓임 최고예요 입맛 없을 때 좋아요 간편해서 좋아요 더 다양한 상품을 컬리에서 구매할 수 있었으면 좋겠어요


Filtered Grouped DataFrame (excluding '활용 방법'):


,category,review
0,가격,가격은 음 착하지는 않습니다 가격 대비 좋네요 그것만 개선되면 자주 이용할 거 같아요 재구매는 하지 않을 듯요 진한 맛이 나라 17000원에 한 끼 아깝지 않게 먹었는데 가격대가 있어도 이 제품만 구매하게 되요 홈쇼핑 판매하는 거랑은 비교 불가 입니다 가격이 비싸서 좀 아쉽습니다 비싸도 최고네요
1,맛/향/풍미,하트찜꽁 쫀득한 도가니탕 맛있네요 국 간은 싱거운데 소금 간 하면 딱이에요 기름 너무 많아서 느끼함 믿고 먹는 설성 목장이예요 소금과 첨가물 안 들어 있어서 좋아요 입에 짝 달라붙는 맛 도가니탕 중 최고입니다 국물 진하고 맛 잇어요 맛있어요 고기 살이 까매요 기름 심해서 버렸다고 비싸도 제대로 집에서 요리된 듯 맛이라 꾸준하게 주문해요 국물 맛은 제일 맛있어요 고기에 기름이 많아요 파 송송에 후추 넣어서 먹으니 맛나요 입이 짧아 지방같이 물컹거리는 부분은 먹지 않는데 누린내가 너무 심합니다 무난했어요 기름은 좀 있지만 설성 목장 제품 좋아합니다 첨가물 없이 깔끔해서요 맛은 어떤 양념도 첨가하지 않은 순수한 탕 맛이라 맘에 들지만 고기는 많이 우려서인지 맛이 안 느껴지고 갑자기 묽어졌어요 이렇게 물 탄 맛이면 앞으로는 시킬 이유가 없겠어요 왜 맛이 묽어졌는지 궁금합니다 엄마가 제일 좋아하는 도가니탕입니다 이게 제일 부드럽고 진하면서 맛있다고 하시네요 설성 목장 도가니탕이 최고입니다 도가니탕이 제일 맛있다고 하셔서 육수가 맑고 구수하다는 것입니다 맛은 좋습니다 나트륨 낮아 주문해요 설성 목장 도가니탕 한우라 좋고 국물이 깔끔해요 국물도 깔끔 너무 맛있어요 맛도 좋아요 믿고 먹는 설성 목장입니다 뭘 먹어도 실패가 없네요 사골 스틱도 먹어 보니까 너무 편하고 맛있던데 부모님이 제일 좋아하십니다 자극적이지 않고 판매하는 특유의 그런 맛이 안 난다고 좋아하세요 설성 목장 도가니탕 정말 맛있었요 도가니탕으로 떡국 끓여 먹었는데 맛있어요
3,양,무 항생제인데 도가니도 꽤 들어 있고 도가니보다 부속물이 많고 도가니가 양이 적긴한데 안에 내용물이 많이 들어 있어서 도가니가 많이 들어 있는 줄 알았는데 고기 몇 점과 지방 덩이네요 건더기가 엄청 많다고 느껴지진 않았고 도가니도 조금밖에 안 들어가서 아쉬웠어요 고기와 도가니가 적당히 들어 있어요 양이 적은 사람은 둘이서 먹어도 부족하지 않아요 고기도 도가니도 많아 넘 좋아요 양은 작아도 고기도 듬뿍 도가니도 많고
4,포장/배송,하나는 멀쩡한데 하나는 누가 일부러 뜯어 놓은 거마냥 저걸 어떻게 먹어요 칼로 난도질해 놓은 것도 아니고


In [ ]:
summary_data = []

# Loop through the grouped data (which is already filtered by a single product_id)
for _, row in grouped_data2.iterrows():
    category = row['category']
    combined_reviews = row['review']

    # Prepare the initial summarization prompt
    summary_prompt = f"""
    당신은 상품에 대한 고객의 리뷰를 요약해서 소비자에게 정보를 제공하는 소비자 분석가입니다.

    **지시사항:**
    리뷰: {combined_reviews}\n
    이 리뷰들을 요약해주세요.
    많이 나오는 내용을 중심으로 요약해주세요.
    요약한 문장을 한 번 더 매끄러운 문장으로 만들어 주세요.

    **제한사항:**
    브랜드 이름은 넣지 마세요.
    중복되는 말과 리뷰, 요약이라는 단어를 삭제해 주세요.

    **예시:**
    {example}
    """
    inputs = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n{summary_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>\n{combined_reviews}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"

    # First pass: Generate the initial summary
    outputs = model.generate(**tokenizer(inputs,
                             return_tensors="pt").to(device),
                             max_new_tokens=500,
                             pad_token_id=tokenizer.eos_token_id,
                             repetition_penalty=1.2
                             )
    initial_summary = extract_assistant_text(outputs)

    # # Save the final summary along with sentiment
    # summary_data.append([test_product_id, category, initial_summary])

    # Second pass: Condense the summary into one sentence
    rephrase_prompt = f"""
    당신은 문장을 간결하게 만드는 전문가입니다.

    **지시사항:**
    다음 문장을 요약해주세요.
    리뷰, 요약이라는 단어를 삭제해 주세요.
    말투는 존댓말이어야 합니다.:
    {initial_summary}
    """
    rephrase_inputs = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n{rephrase_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>\n{initial_summary}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"

    # Generate the final rephrased summary
    rephrase_outputs = model.generate(**tokenizer(inputs,
                             return_tensors="pt").to(device),
                             max_new_tokens=200,
                             pad_token_id=tokenizer.eos_token_id
                             )
    final_summary = extract_assistant_text(rephrase_outputs)

    # Save the final summary along with sentiment
    summary_data.append([test_product_id, category, final_summary])


# Create a DataFrame for the summarized data
summary_df = pd.DataFrame(summary_data, columns=['product_id', 'category', 'summary'])

# Display the result
summary_df

,product_id,category,summary
0,5013198,가격,"이 제품은 가격대가 있어도 만족스러운 가성비로 인기가 많고, 맛이 좋다는 고객들의 평이 많아요. 가격이 비싸다는 의견도 있지만, 가격 대비 만족스러운 제품으로 편리하고 맛있는 제품입니다."
1,5013198,맛/향/풍미,"소비자들은 이 도가니탕을 믿고 먹을 수 있는 이유는 소금과 첨가물이 들어가지 않아 깔끔하고, 기름이 많지만 맛이 있고, 국물이 진하고 맛이 있다는 점입니다. 또한, 고기 살이 까맣고, 파 송송에 후추를 넣으면 더 맛나다는 팁도 있습니다."
2,5013198,양,"고기와 도가니가 적당히 들어 있는 제품이에요. 도가니보다 부속물이 많고, 양이 적은 사람은 둘이서 먹어도 부족하지 않아요."
3,5013198,포장/배송,"이 제품은 일부 고객이 만족스럽게 사용했지만, 다른 고객은 일부러 뜯어놓은 것처럼 보이는 문제가 있다는 의견이 있습니다."


In [ ]:
summary_recipe_data = []

# Loop through the grouped data (which is already filtered by a single product_id)
for _, row in grouped_recipe.iterrows():
    category = row['category']
    combined_reviews = row['review']

    # Prepare the initial summarization prompt
    summary_prompt = f"""
    당신은 상품에 대한 고객의 리뷰를 요약해서 소비자에게 정보를 제공하는 소비자 분석가입니다.

    **지시사항:**
    리뷰: {combined_reviews}\n
    이 리뷰들은 상품의 활용방법에 대한 내용입니다.
    이 리뷰들을 요약해주세요.
    많이 나오는 내용을 중심으로 요약해주세요.
    요약한 문장을 한 번 더 매끄러운 문장으로 만들어 주세요.

    **제한사항:**
    브랜드 이름은 넣지 마세요.
    리뷰, 요약이라는 단어를 삭제해 주세요.

    **예시:**
    {example}
    """
    inputs = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n{summary_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>\n{combined_reviews}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"

    # First pass: Generate the initial summary
    outputs = model.generate(**tokenizer(inputs,
                             return_tensors="pt").to(device),
                             max_new_tokens=500,
                             pad_token_id=tokenizer.eos_token_id,
                             repetition_penalty=1.2
                             )
    initial_summary = extract_assistant_text(outputs)

    # # 프롬프트 하나만 할 때
    # summary_recipe_data.append([test_product_id, category, initial_summary])

    # Second pass: Condense the summary into one sentence
    rephrase_prompt = f"""
    당신은 문장을 간결하게 만드는 전문가입니다.

    **지시사항:**
    다음 문장을 요약해주세요.
    리뷰, 요약이라는 단어를 삭제해 주세요.
    말투는 존댓말이어야 합니다.:
    {initial_summary}
    """
    rephrase_inputs = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n{rephrase_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>\n{initial_summary}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"

    # Generate the final rephrased summary
    rephrase_outputs = model.generate(**tokenizer(rephrase_inputs, return_tensors="pt").to(device), max_new_tokens=200, pad_token_id=tokenizer.eos_token_id)
    final_summary = extract_assistant_text(rephrase_outputs)

    # Save the final summary along with sentiment
    summary_recipe_data.append([test_product_id, category, final_summary])


# Create a DataFrame for the summarized data
summary_recipe_df = pd.DataFrame(summary_recipe_data, columns=['product_id', 'category', 'summary_reicpe'])

# Display the result
summary_recipe_df

,product_id,category,summary_reicpe
0,5013198,활용 방법,"간단한 조리 방법으로 맛이 없을 때 최고의 선택이며, 시간을 많이 절약할 수 있어 편리한 식품입니다."


In [ ]:
summary_df.to_csv("/content/drive/MyDrive/딥다이브 프로젝트/요약_상품별.csv", index=False, encoding="utf-8-sig")